In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric.nn as pyg_nn
from torch_geometric.transforms import RandomLinkSplit
import pandas as pd
import os
from tqdm import tqdm

/home/acaia/Analysa/Analysa/.analysa/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class Encoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels, aggr='mean'):
        super(Encoder, self).__init__()
        self.conv1 = pyg_nn.SAGEConv(in_channels, 2 * out_channels, aggr=aggr) # Eventually change layer type
        self.batch1 = pyg_nn.BatchNorm(2 * out_channels)
        self.drop1 = nn.Dropout(p=0.2)
        self.conv2 = pyg_nn.SAGEConv(2 * out_channels, out_channels,  aggr=aggr) # Eventually change layer type
        self.batch2 = pyg_nn.BatchNorm(out_channels)

    def forward(self, x, edge_index): # Modify to adapt to the dataset complexity
        x = self.conv1(x, edge_index)
        x = self.batch1(x)
        x = self.drop1(x)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = self.batch2(x)

        return x
    
class Decoder(nn.Module):
    def __init__(self, in_channels):
        super(Decoder, self).__init__()
        self.linear = nn.Linear(in_channels, 1)

    def forward(self, z, edge_index):
        # By default, it computes the inner product of the node embeddings
        adj = torch.matmul(z[edge_index[0]], z[edge_index[1]].t())
        return adj[edge_index[0], edge_index[1]]
    
# Defining traing and evaluation functions
def train(epoch):
    model.train()
    optimizer.zero_grad()
    z = model.encode(x, train_data.edge_index.to(device))
    loss = model.recon_loss(z, train_data.pos_edge_label_index.to(device))
    loss.backward()
    optimizer.step()

def test(pos_edge_index, neg_edge_index):
    model.eval()
    # Encode the node features to get the embeddings
    with torch.no_grad():
        z = model.encode(x, train_data.edge_index.to(device))
    
    # Compute the reconstruction loss
    return model.test(z, pos_edge_index.to(device), neg_edge_index.to(device))
    

In [3]:
# Model definition
LATENT_DIM = 32
data_path = os.path.join('..', 'data', 'graph_data.pt')
dataset = torch.load(data_path, weights_only=False)
data = dataset['data']

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# Instantiate the customized encoder in GAE model
transform = RandomLinkSplit(is_undirected=True, split_labels=True,add_negative_train_samples=True)
train_data, val_data, test_data = transform(data)

model = pyg_nn.GAE(Encoder(data.num_features, LATENT_DIM)).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
x = train_data.x.to(device)

Using device: cuda


In [5]:
# Training loop
for epoch in tqdm(range(1, 1001)):
    loss = train(epoch)
    # AUC tells how well the model is able to distinguish real nodes from fake ones
    # AP - Average Precision is the complementary metric of AUC 
    auc, ap = test(test_data.pos_edge_label_index, test_data.neg_edge_label_index)


print('Final Evaluation: AUC: {:.4f}, AP: {:.4f}'.format(auc, ap))

100%|██████████| 1000/1000 [00:25<00:00, 38.79it/s]

Final Evaluation: AUC: 0.7458, AP: 0.7709


In [6]:
model.eval()
z = model.encode(x, train_data.edge_index.to(device))
emb_sample = z[0]
print(f'Number of nodes: {z.shape[0]}')
print(f'Number of edges: {train_data.edge_index.shape[1]}')
print(f'Embedding sample: {z}')

Number of nodes: 4250
Number of edges: 20102
Embedding sample: tensor([[-0.0148,  0.3190, -0.2321,  ...,  0.2931, -0.2364, -0.0416],
        [ 0.4805, -0.4786, -0.2518,  ...,  0.6031, -0.8688, -0.7159],
        [-0.0649, -3.2233, -7.3970,  ...,  1.2354, -7.0063,  7.7511],
        ...,
        [-6.3847,  4.2103,  2.0474,  ...,  6.0607,  4.0489, -1.4764],
        [-0.1624, -3.6282, -8.8758,  ...,  1.5324, -7.7481,  9.3953],
        [-6.4098,  4.2184,  2.0403,  ...,  6.0689,  4.0344, -1.5072]],
       device='cuda:0', grad_fn=<NativeBatchNormBackward0>)


In [7]:
mapping_list = dataset['idx2user']
embeddings = z
embeddings_np = embeddings.cpu().detach().numpy()

df = pd.DataFrame(embeddings_np)
df.insert(loc=0, column='id', value=range(len(embeddings_np)))
df.insert(loc=1, column='user_id', value= df['id'].map(mapping_list))

df.to_csv('user_embeddings.csv', index=False)